DATA CLEANING

In [71]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [72]:
df = pd.read_csv("/Users/tharunnesh/Downloads/Phase2/data/covid_deaths_linelist.csv")

In [73]:
df.shape

(37351, 15)

In [74]:
df.columns

Index(['date', 'date_announced', 'date_positive', 'date_dose1', 'date_dose2',
       'date_dose3', 'brand1', 'brand2', 'brand3', 'state', 'age', 'male',
       'bid', 'malaysian', 'comorb'],
      dtype='object')

In [75]:
df.info()
df.isna().sum().sort_values(ascending=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37351 entries, 0 to 37350
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   date            37351 non-null  object
 1   date_announced  37351 non-null  object
 2   date_positive   37351 non-null  object
 3   date_dose1      14876 non-null  object
 4   date_dose2      9276 non-null   object
 5   date_dose3      1543 non-null   object
 6   brand1          14876 non-null  object
 7   brand2          9276 non-null   object
 8   brand3          1543 non-null   object
 9   state           37351 non-null  object
 10  age             37351 non-null  int64 
 11  male            37351 non-null  int64 
 12  bid             37351 non-null  int64 
 13  malaysian       37351 non-null  int64 
 14  comorb          37351 non-null  int64 
dtypes: int64(5), object(10)
memory usage: 4.3+ MB


date_dose3        35808
brand3            35808
date_dose2        28075
brand2            28075
date_dose1        22475
brand1            22475
date                  0
date_announced        0
date_positive         0
state                 0
age                   0
male                  0
bid                   0
malaysian             0
comorb                0
dtype: int64

In [76]:
date_cols = ["date","date_announced","date_positive","date_dose1","date_dose2","date_dose3"]

for c in date_cols:
    df[c] = pd.to_datetime(df[c], errors="coerce")

In [77]:
df[date_cols].isna().sum()

date                  0
date_announced        0
date_positive         0
date_dose1        22475
date_dose2        28075
date_dose3        35808
dtype: int64

In [78]:
df.duplicated().sum()

np.int64(20)

In [79]:
df = df.drop_duplicates().reset_index(drop=True)

In [80]:
binary_cols = ["male","bid","malaysian","comorb"]
for c in binary_cols:
    print(c, sorted(df[c].unique()))

male [np.int64(0), np.int64(1)]
bid [np.int64(0), np.int64(1)]
malaysian [np.int64(0), np.int64(1)]
comorb [np.int64(0), np.int64(1)]


In [81]:
df["gap_positive_to_death"] = (df["date"] - df["date_positive"]).dt.days
df["gap_death_to_announced"] = (df["date_announced"] - df["date"]).dt.days

(df["gap_positive_to_death"] < 0).sum(), (df["gap_death_to_announced"] < 0).sum()

(np.int64(0), np.int64(0))

FEATURE ENGINEERING

In [82]:
def vaccination_status(row):
    if pd.isna(row["date_dose1"]):
        return "Unvaccinated"
    if pd.isna(row["date_dose2"]):
        return "Partial (1 dose)"
    if pd.isna(row["date_dose3"]):
        return "Fully vaccinated (2 doses)"
    return "Booster (3 doses)"

df["vax_status"] = df.apply(vaccination_status, axis=1)
df["vax_status"].value_counts()

vax_status
Unvaccinated                  22457
Fully vaccinated (2 doses)     7733
Partial (1 dose)               5599
Booster (3 doses)              1542
Name: count, dtype: int64

In [83]:
bins = [0, 17, 29, 39, 49, 59, 69, 200]
labels = ["0-17","18-29","30-39","40-49","50-59","60-69","70+"]

df["age_group"] = pd.cut(df["age"], bins=bins, labels=labels, include_lowest=True)

df["age_group"] = pd.cut(df["age"], bins=bins, labels=labels, right=True, include_lowest=True)
df["age_group"].value_counts().sort_index()

age_group
0-17       207
18-29      815
30-39     2490
40-49     4708
50-59     6685
60-69     8522
70+      13904
Name: count, dtype: int64

In [84]:
df["last_dose_date"] = df[["date_dose1","date_dose2","date_dose3"]].max(axis=1)
df["days_last_dose_to_death"] = (df["date"] - df["last_dose_date"]).dt.days

In [85]:
brand_cols = ["brand1","brand2","brand3"]
for c in brand_cols:
    df[c] = df[c].astype(str).str.strip()
    df.loc[df[c].isin(["nan","None","NaT",""]), c] = np.nan

In [86]:
df["primary_brand"] = df["brand1"]
df["primary_brand"].value_counts(dropna=False).head(20)

primary_brand
NaN                 22457
Sinovac              8148
Pfizer               5571
AstraZeneca          1071
Cansino                68
Pending VMS sync        7
Sinopharm               7
Moderna                 2
Name: count, dtype: int64

In [87]:
df.to_csv("/Users/tharunnesh/Downloads/Phase2/outputs/covid_deaths_cleaned.csv", index=False)